In [5]:
# Import necessary libraries for managing datasets, image processing, and modeling.

# Datasets and XML parsing
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import h5py

# Image preprocessing
from skimage import feature
import skimage
from sklearn.feature_extraction.image import extract_patches_2d
from PIL import Image, ImageFilter, ImageEnhance, ImageDraw, ImageFont
import imgaug.augmenters as iaa
from skimage.color import rgb2gray
from skimage.io import imread
from skimage.transform import resize, pyramid_gaussian

#Generic libraries
import numpy as np
import matplotlib.pyplot as plt
import argparse
import random
import os

# Model & Training
from sklearn.svm import SVC
import pickle
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV

#### Here we define some paths to jpeg and xml files

*for generate xml with all information about class and boxes coordinates LabelImg where used*

In [17]:
all_jpeg = glob.glob('cumulative_jpeg/'+'*.jpeg')
all_xml = glob.glob('cumulative_xml/'+'*.xml')

In [18]:
# Iterate through all XML files and build a tree object for each one.
# The last tree object in the loop will be the one retained.
for t in all_xml:
    tree = ET.parse(t)

# Get the root element of the tree, which represents the top-level element in the XML file.
root = tree.getroot()

# Print the XML tree (this will display the tree's root element and its structure).
print(tree)

# Initialize an empty list to store the tags present in the XML files.
child_tags = []

# Iterate through the children of the root element and extract their tag names.
# Append these tag names to the child_tags list.
for child in root:
    childtag = child.tag
    child_tags.append(childtag)

# Print the list of extracted child tags from the XML files.
print(child_tags)

['folder', 'filename', 'path', 'source', 'size', 'segmented', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object']


In [19]:
# Iterate through the elements within the "size" tag.
# This tag often contains information about the width and height of the images.
for s in root.find('size'):
    # Print the individual elements within the "size" tag.
    print(s)

<Element 'width' at 0x12fa09d60>
<Element 'height' at 0x12fa09d10>
<Element 'depth' at 0x12fa09cc0>


#### With this code we can extract bouding boxes coordinates from the xml files

In [20]:
def extract_bounding_boxes(xml_files):
    bounding_boxes = []

    for t in xml_files:
        tree = ET.parse(t)
        root = tree.getroot()

        for obj in root.findall('object'):
            name = obj.find('name').text
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            bounding_box_info = {'name': name, 'xmin': xmin,
                                 'ymin': ymin, 'xmax': xmax,
                                 'ymax': ymax}
            bounding_boxes.append(bounding_box_info)

    return bounding_boxes

# Call the function with list of XML files
bounding_boxes = extract_bounding_boxes(all_xml)

# Print the extracted bounding box information as an example to show how it works
for index, box in enumerate(bounding_boxes[:2], start=1):
    print(f"Bounding Box {index} - Object: {box['name']}")
    print(f"xmin: {box['xmin']}, ymin: {box['ymin']}, xmax: {box['xmax']}, ymax: {box['ymax']}")

Bounding Box 1 - Object: face
xmin: 15, ymin: 14, xmax: 79, ymax: 72
Bounding Box 2 - Object: face
xmin: 55, ymin: 55, xmax: 98, ymax: 108
